In [9]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import random


In [10]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])
a = open('processed.cleveland.data', 'r')
c = a.readlines()
random.shuffle(c) #shuffle data before using
n_val = []
n_keys = []
d = []
for elem in c:
    elem = elem[:-1]
    elem = list(map(float,elem.split(',')))
    n_val.append(elem[:-1])
    n_keys.append(elem[-1]) #mb ad
rot = [[] for i in range(len(n_val[0]))]
mean = []
std = []
for i in range(len(n_val)):
    for j in range(len(n_val[i])):
        rot[j].append(n_val[i][j])
#print(len(rot[0]))
for elem in rot:
    #print(elem)
    new_elem = np.array(elem)
    #print(np.std(new_elem))
    #print(new_elem.shape)
    std.append(np.std(new_elem))
    mean.append(np.mean(new_elem))
    #elem = [int(elem[i]*10) for i in range(len(elem))] 
for i in range(len(rot)):
    rot[i] = [(rot[i][j] - mean[i])/std[i] for j in range(len(elem))]
#print(rot[0])
rot = np.array(rot)
n_val = rot.T     #all of the previous 20 lines normalises our array


#true-false
for i in range(len(n_keys)):
    if n_keys[i] > 0:
        n_keys[i] = 1
print(n_keys)

[0.0, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 1, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 1, 1, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1, 1, 0.0, 1, 0.0, 1, 1, 1, 1, 1, 0.0, 1, 1, 0.0, 1, 1, 0.0, 0.0, 1, 1, 1, 0.0, 0.0, 1, 0.0, 1, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 1, 0.0, 1, 1, 0.0, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [11]:
#73 - test size
#230 - train size
t = 230
te = 73
a = random.sample(range(303), t)
val, keys, te_val, te_keys = [],[],[],[]
for i in range(len(n_val)):
    if i in a:
        val.append(n_val[i])
        keys.append(n_keys[i])
    else:
        te_val.append(n_val[i])                       #train-test 
        te_keys.append(n_keys[i])

val = np.array(val)
keys = np.array(keys)
#val = val.astype(float)
#keys = keys.astype(float)

te_val = np.array(te_val)
te_keys = np.array(te_keys)
#keys.shape
val.shape



#now we need to create a cycle for bootstrap


(224, 13)

In [12]:
number_of_models = 100
voting_list = []
for trap in range(number_of_models):
    trap_val, trap_keys = [], []
    for i in range(len(val)):
        ind = random.randrange(len(val))
        trap_val.append(val[ind])
        trap_keys.append(keys[ind])
    #print(len(trap_keys))
    input_size = 13
    hidden_sizes = [128, 128, 64]
    output_size = 2

    model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[1], output_size),
                          nn.LogSoftmax(dim=1))
    #print(model) #model with 3 layers, first 2 with relu, last with logistic curve
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
    time0 = time()
    epochs = 30
    for e in range(epochs):
        running_loss = 0
        for i in range(len(trap_val)):
            # Training pass
            optimizer.zero_grad()
            temp_1 = [trap_val[i]]
            temp0 = np.array(temp_1)
            temp = torch.from_numpy(temp0)
            temp1 = temp.float()
            #print(temp1)
            output = model(temp1)    #DOES NOT WORK. MODEL RETURNS NAN TENSOR
            #print(output)
            k = [trap_keys[i]]
            k = np.array(k)
            k = torch.from_numpy(k)
            k = k.long()
            loss = criterion(output, k)

            #This is where the model learns by backpropagating
            loss.backward()

            #And optimizes its weights here
            optimizer.step()

            #print(loss.item())
            running_loss += loss.item()
        else:
            print("Epoch {} - Training loss: {}".format(e, running_loss/len(val)))

    print("\nTraining Time (in minutes) =",(time()-time0)/60)
    
    model_answers = []
    for i in range(len(te_val)):
        temp_1 = [te_val[i]]
        temp0 = np.array(temp_1)
        temp = torch.from_numpy(temp0)
        temp1 = temp.float()
        with torch.no_grad():
            logps = model(temp1)
        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        model_answers.append(pred_label)
    voting_list.append(model_answers)


Epoch 0 - Training loss: 0.4580038514156643
Epoch 1 - Training loss: 0.2938761853023867
Epoch 2 - Training loss: 0.19329038420929529
Epoch 3 - Training loss: 0.13755141226863138
Epoch 4 - Training loss: 0.1012311753653531
Epoch 5 - Training loss: 0.1065338535724102
Epoch 6 - Training loss: 0.06869119976931708
Epoch 7 - Training loss: 0.04308144343653567
Epoch 8 - Training loss: 0.030696138160908295
Epoch 9 - Training loss: 0.05010927762158628
Epoch 10 - Training loss: 0.05592733870944185
Epoch 11 - Training loss: 0.016114294982913707
Epoch 12 - Training loss: 0.004995265628291667
Epoch 13 - Training loss: 0.003351633472599242
Epoch 14 - Training loss: 0.002490785192460075
Epoch 15 - Training loss: 0.0019904452426894154
Epoch 16 - Training loss: 0.0016941923254975805
Epoch 17 - Training loss: 0.0014358600353190365
Epoch 18 - Training loss: 0.0012643303245515167
Epoch 19 - Training loss: 0.001111240891088272
Epoch 20 - Training loss: 0.0009889239783925482
Epoch 21 - Training loss: 0.0009

Epoch 20 - Training loss: 0.0011853962656846065
Epoch 21 - Training loss: 0.0010965495974752301
Epoch 22 - Training loss: 0.001012520724423435
Epoch 23 - Training loss: 0.0009404511251507637
Epoch 24 - Training loss: 0.0008822970733008972
Epoch 25 - Training loss: 0.0008294479306290649
Epoch 26 - Training loss: 0.0007753033053489654
Epoch 27 - Training loss: 0.0007362964077246888
Epoch 28 - Training loss: 0.0006941245874349125
Epoch 29 - Training loss: 0.000658084431672336

Training Time (in minutes) = 0.13211137851079305
Epoch 0 - Training loss: 0.44411776858188595
Epoch 1 - Training loss: 0.25575402644655504
Epoch 2 - Training loss: 0.1759513208046022
Epoch 3 - Training loss: 0.11483761016881187
Epoch 4 - Training loss: 0.06361728412254651
Epoch 5 - Training loss: 0.03652444531276552
Epoch 6 - Training loss: 0.023436981352873514
Epoch 7 - Training loss: 0.014392157241224193
Epoch 8 - Training loss: 0.011410241774190937
Epoch 9 - Training loss: 0.1519793543164363
Epoch 10 - Training l

Epoch 9 - Training loss: 0.05547064404463845
Epoch 10 - Training loss: 0.03690617489298861
Epoch 11 - Training loss: 0.03861875735881754
Epoch 12 - Training loss: 0.025972520595011366
Epoch 13 - Training loss: 0.017226165951263046
Epoch 14 - Training loss: 0.01994844412264109
Epoch 15 - Training loss: 0.010778914447672523
Epoch 16 - Training loss: 0.005692072920575894
Epoch 17 - Training loss: 0.0034158482237495974
Epoch 18 - Training loss: 0.002615510440086455
Epoch 19 - Training loss: 0.0021890524671240703
Epoch 20 - Training loss: 0.0018912552819538853
Epoch 21 - Training loss: 0.0016846630309156816
Epoch 22 - Training loss: 0.0015006931613303909
Epoch 23 - Training loss: 0.0013556745021691288
Epoch 24 - Training loss: 0.0012435985988733364
Epoch 25 - Training loss: 0.0011365639983692713
Epoch 26 - Training loss: 0.0010597614985212406
Epoch 27 - Training loss: 0.0009848327108784424
Epoch 28 - Training loss: 0.0009216434003085361
Epoch 29 - Training loss: 0.0008644019956452238

Train

Epoch 28 - Training loss: 0.0005980266208712567
Epoch 29 - Training loss: 0.0005773344151631031

Training Time (in minutes) = 0.16401409705479938
Epoch 0 - Training loss: 0.4547988730285267
Epoch 1 - Training loss: 0.29356433153407124
Epoch 2 - Training loss: 0.23034764419259904
Epoch 3 - Training loss: 0.18015359715393434
Epoch 4 - Training loss: 0.13543065981962904
Epoch 5 - Training loss: 0.09424700305274973
Epoch 6 - Training loss: 0.06323564401338452
Epoch 7 - Training loss: 0.03663692062738476
Epoch 8 - Training loss: 0.021603135469547046
Epoch 9 - Training loss: 0.013640855636793927
Epoch 10 - Training loss: 0.00880101970141034
Epoch 11 - Training loss: 0.0058445533658746385
Epoch 12 - Training loss: 0.0045805628572528345
Epoch 13 - Training loss: 0.0035686998712243234
Epoch 14 - Training loss: 0.0029667239288143792
Epoch 15 - Training loss: 0.0024724842965231997
Epoch 16 - Training loss: 0.0021305021124539613
Epoch 17 - Training loss: 0.0018595416844506194
Epoch 18 - Training l

Epoch 17 - Training loss: 0.001516764100304911
Epoch 18 - Training loss: 0.0013349500233285547
Epoch 19 - Training loss: 0.0011937941799946244
Epoch 20 - Training loss: 0.0010767478478223807
Epoch 21 - Training loss: 0.000981550941616576
Epoch 22 - Training loss: 0.0009006712597348722
Epoch 23 - Training loss: 0.0008336471657581779
Epoch 24 - Training loss: 0.0007751107764220055
Epoch 25 - Training loss: 0.0007238141161824022
Epoch 26 - Training loss: 0.0006784682770460483
Epoch 27 - Training loss: 0.0006385864112456055
Epoch 28 - Training loss: 0.0006031377602100056
Epoch 29 - Training loss: 0.0005710803280022539

Training Time (in minutes) = 0.14461245934168498
Epoch 0 - Training loss: 0.43322638185600226
Epoch 1 - Training loss: 0.29283991419607347
Epoch 2 - Training loss: 0.22890665513725952
Epoch 3 - Training loss: 0.16823297025588704
Epoch 4 - Training loss: 0.12131493035782774
Epoch 5 - Training loss: 0.09178798517894189
Epoch 6 - Training loss: 0.06793432500499051
Epoch 7 - Tra

...

Epoch 11 - Training loss: 0.013443326926976695
Epoch 12 - Training loss: 0.009635463307486372
Epoch 13 - Training loss: 0.007714965068609437
Epoch 14 - Training loss: 0.00608839189545618
Epoch 15 - Training loss: 0.004570291715058643
Epoch 16 - Training loss: 0.0038422192668448357
Epoch 17 - Training loss: 0.002851865211905042
Epoch 18 - Training loss: 0.0024408418908787866
Epoch 19 - Training loss: 0.002067040214525909
Epoch 20 - Training loss: 0.0017846517962176098
Epoch 21 - Training loss: 0.0015553204560886538
Epoch 22 - Training loss: 0.0013831340554380908
Epoch 23 - Training loss: 0.0012685993614044591
Epoch 24 - Training loss: 0.0011358254658882913
Epoch 25 - Training loss: 0.0010433550740318914
Epoch 26 - Training loss: 0.0009669263933909418
Epoch 27 - Training loss: 0.0008901543750460103
Epoch 28 - Training loss: 0.0008325605440051247
Epoch 29 - Training loss: 0.0007748086737696511

Training Time (in minutes) = 0.1323947270711263
Epoch 0 - Training loss: 0.42335424519686576
Ep

Training Time (in minutes) = 0.1517797311147054
Epoch 0 - Training loss: 0.4529197687723873
Epoch 1 - Training loss: 0.30545752946246857
Epoch 2 - Training loss: 0.23273926006413603
Epoch 3 - Training loss: 0.17691781851683736
Epoch 4 - Training loss: 0.12333983807191576
Epoch 5 - Training loss: 0.08944386825634353
Epoch 6 - Training loss: 0.08339928954611195
Epoch 7 - Training loss: 0.04680282769110595
Epoch 8 - Training loss: 0.03841698327469371
Epoch 9 - Training loss: 0.026598671940887338
Epoch 10 - Training loss: 0.016040158348536022
Epoch 11 - Training loss: 0.0128396265127633
Epoch 12 - Training loss: 0.009432712147583868
Epoch 13 - Training loss: 0.008342467975313557
Epoch 14 - Training loss: 0.0057632018942788766
Epoch 15 - Training loss: 0.004440977548299895
Epoch 16 - Training loss: 0.003397137713357098
Epoch 17 - Training loss: 0.0027986993982202107
Epoch 18 - Training loss: 0.002327577221279028
Epoch 19 - Training loss: 0.0019653710736145286
Epoch 20 - Training loss: 0.001

Epoch 19 - Training loss: 0.0010343174828893861
Epoch 20 - Training loss: 0.0009459141210793451
Epoch 21 - Training loss: 0.0008524499148328932
Epoch 22 - Training loss: 0.0007880900307022376
Epoch 23 - Training loss: 0.0007238617782921532
Epoch 24 - Training loss: 0.000676176126237303
Epoch 25 - Training loss: 0.0006291131792578611
Epoch 26 - Training loss: 0.000590964207717315
Epoch 27 - Training loss: 0.0005540736363676453
Epoch 28 - Training loss: 0.0005247794526297422
Epoch 29 - Training loss: 0.000494929082696676

Training Time (in minutes) = 0.14809608062108356
Epoch 0 - Training loss: 0.4838554984486499
Epoch 1 - Training loss: 0.31715660200539525
Epoch 2 - Training loss: 0.24774497352928343
Epoch 3 - Training loss: 0.20299411740736772
Epoch 4 - Training loss: 0.16603514306985123
Epoch 5 - Training loss: 0.12638724783374777
Epoch 6 - Training loss: 0.08860136789208596
Epoch 7 - Training loss: 0.07505245343593235
Epoch 8 - Training loss: 0.06984453897144552
Epoch 9 - Training lo

Epoch 7 - Training loss: 0.08074353779012082
Epoch 8 - Training loss: 0.06147020305535796
Epoch 9 - Training loss: 0.044182896043222515
Epoch 10 - Training loss: 0.029601415180864437
Epoch 11 - Training loss: 0.021126954122972323
Epoch 12 - Training loss: 0.01466518026888708
Epoch 13 - Training loss: 0.01279387583921546
Epoch 14 - Training loss: 0.010171756786458699
Epoch 15 - Training loss: 0.00772705378134069
Epoch 16 - Training loss: 0.0057415762856917974
Epoch 17 - Training loss: 0.004294572835454139
Epoch 18 - Training loss: 0.003490630063547741
Epoch 19 - Training loss: 0.0029177216015389617
Epoch 20 - Training loss: 0.0025151152488578354
Epoch 21 - Training loss: 0.002231224631388188
Epoch 22 - Training loss: 0.0019824223775669453
Epoch 23 - Training loss: 0.0017892156928143801
Epoch 24 - Training loss: 0.0016180469448593574
Epoch 25 - Training loss: 0.0014881990864605843
Epoch 26 - Training loss: 0.0013726907688336493
Epoch 27 - Training loss: 0.0012707251099924402
Epoch 28 - T

Epoch 27 - Training loss: 0.0005963661525390597
Epoch 28 - Training loss: 0.0005539038274936038
Epoch 29 - Training loss: 0.0005125350874899404

Training Time (in minutes) = 0.13552833398183187
Epoch 0 - Training loss: 0.4433367838815424
Epoch 1 - Training loss: 0.2966929006337133
Epoch 2 - Training loss: 0.23383180660825278
Epoch 3 - Training loss: 0.17392842907188008
Epoch 4 - Training loss: 0.12493188678601871
Epoch 5 - Training loss: 0.10120751286893305
Epoch 6 - Training loss: 0.07023683701690595
Epoch 7 - Training loss: 0.169651550292192
Epoch 8 - Training loss: 0.04390620876196822
Epoch 9 - Training loss: 0.02046131746437263
Epoch 10 - Training loss: 0.008839946405478345
Epoch 11 - Training loss: 0.004899980699300929
Epoch 12 - Training loss: 0.003503415211755488
Epoch 13 - Training loss: 0.0027057912147267415
Epoch 14 - Training loss: 0.0021789198548882727
Epoch 15 - Training loss: 0.001841017637044251
Epoch 16 - Training loss: 0.0015749165441487634
Epoch 17 - Training loss: 0.

In [29]:
#here we are going out of the cycle and compare results via voting




correct_count, all_count = 0, 0    
norm_c, norm_true = 0,0
dis_c, dis_true = 0,0

for i in range(len(te_val)):
    sum_prediction = 0
    for j in range(len(voting_list)):
        sum_prediction += voting_list[j][i]
    if sum_prediction >= number_of_models//2.7:
        prediction = 1
    else:
        prediction = 0
    true_label = te_keys[i]
    if(true_label == prediction):
        correct_count += 1
        if true_label == 0:
            norm_true += 1
        else:
            dis_true += 1
    if true_label == 0:
        norm_c += 1
    else:
        dis_c += 1        
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 73

Model Accuracy = 0.8493150684931506


In [30]:
#true-false 80 percent need 90
print("Number Of norm =", norm_c)
print("\nModel Accuracy norm=", (norm_true/norm_c))
print("Number Of dis =", dis_c)
print("\nModel Accuracy dis=", (dis_true/dis_c))

Number Of norm = 40

Model Accuracy norm= 0.85
Number Of dis = 33

Model Accuracy dis= 0.8484848484848485


In [15]:














torch.save(model, './my_mnist_model.pt') 